### Class Decorators

In [1]:
def savings(cls):
    cls.account_type = 'savings'
    return cls

def checking(cls):
    cls.account_type = 'checking'
    return cls

In [5]:
class Account:
    pass

@savings
class Bank1Savings(Account):
    pass

@savings
class Bank2Savings(Account):
    pass

@checking
class Bank1Checking(Account):
    pass

@checking
class Bank2Checking(Account):
    pass

In [3]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'savings'})

In [6]:
Bank2Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': None,
              'account_type': 'checking'})

In [7]:
def account_type(type_):
    def decorator(cls):
        cls.account_type = type_
        return cls
    return decorator

In [8]:
@account_type('Savings')
class Bank1Savings:
    pass

In [9]:
@account_type('Checking')
class Bank1Checking:
    pass

In [10]:
Bank1Savings.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Savings' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Savings' objects>,
              '__doc__': None,
              'account_type': 'Savings'})

In [11]:
Bank1Checking.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Bank1Checking' objects>,
              '__weakref__': <attribute '__weakref__' of 'Bank1Checking' objects>,
              '__doc__': None,
              'account_type': 'Checking'})

In [12]:
def hello(cls):
    cls.hello = lambda self: f'{self} says hello!'
    return cls

In [13]:
@hello
class Person:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

In [14]:
vars(Person)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Person.__init__(self, name)>,
              '__str__': <function __main__.Person.__str__(self)>,
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None,
              'hello': <function __main__.hello.<locals>.<lambda>(self)>})

In [15]:
p = Person("Guido")

In [16]:
p.hello()

'Guido says hello!'